In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
import cufflinks as cf
import plotly
import json
plotly.tools.set_credentials_file(username='jc4673', api_key='dwruizxiu5')

In [ ]:
py.init_notebook_mode(connected=True) # run at the start of every ipython notebook to use plotly.offline
                     # this injects the plotly.js source files into the notebook
cf.go_offline()

In [ ]:
def get_highest_cat(cat_list, rankings):
    rankings = [rankings[cat.strip()]['ranking'] for cat in cat_list]
    return cat_list[rankings.index(min(rankings))]

In [ ]:
ranking_config = "C:/Users/jc4673/Documents/Columbia/nets_wrangle/category_ranking/ranking_config.json"
with open(ranking_config) as f:
    rankings = json.load(f)

In [ ]:
infile_long = r'C:\Users\jc4673\Documents\Columbia\NETS2013_Wrangled\NETS2013_Classifications_Long.txt'
infile_normal = r'C:\Users\jc4673\Documents\Columbia\NETS2013_Wrangled\NETS2013_Classifications.txt'
df_long = pd.read_table(infile_long, chunksize=10**6)
df_normal = pd.read_table(infile_normal, chunksize=10**6)

In [ ]:
#Perform initial counts for first chunk
long_chunk = df_long.get_chunk()
normal_chunk = df_normal.get_chunk()

In [ ]:
beh_counts = normal_chunk.drop_duplicates(subset='DunsNumber', keep='first').groupby('BEH_Class').count()
counts = long_chunk.groupby(['Year', 'Class']).count()['DunsNumber']

In [ ]:
#additively get BEH counts for remaining chunks
for chunk in df_normal:
    beh_counts = beh_counts.add(chunk.drop_duplicates(subset='DunsNumber', keep='first').groupby('BEH_Class').count(),
                                fill_value=0)

In [ ]:
beh_counts.drop('not', inplace=True)

In [ ]:
beh_counts = beh_counts.reset_index(drop=False)
beh_counts = beh_counts[['BEH_Class', 'DunsNumber']]

In [ ]:
func = lambda x:  get_highest_cat(x.split(','), rankings)
beh_counts['BEH_Class'] = beh_counts['BEH_Class'].apply(func)

In [ ]:
beh_counts = beh_counts.groupby('BEH_Class').sum()
beh_counts['DunsNumber'] = beh_counts['DunsNumber'].astype(int)

In [ ]:
beh_counts.columns = ['Counts']

In [ ]:
beh_counts.iplot(kind='bar',
                      title = 'BEH_Category Counts',
                      xTitle = 'Category',
                      yTitle = 'Number of Businesses')

In [ ]:
beh_counts.to_csv('BEH_Category Counts.csv')

In [ ]:
%%time
#additively get longitudinal counts for additional chunks
for chunk in df_long:
    counts = counts.add(chunk.groupby(['Year','Class']).count()['DunsNumber'], fill_value=0)

In [ ]:
counts_newindex = counts.reset_index(name='counts')
dropnot = counts_newindex[counts_newindex.Class != 'not']
counts_pivot = dropnot.pivot('Year','Class','counts').fillna(0).astype(int) #pivot for line graph

In [ ]:
#color array to use for charts later
modcolors = ['red', 'blue', 'green', 'brown', 'aqua', 'violet', 'crimson', 'darkgoldenrod', 'cornflowerblue', 'pink', 'yellow', 
          'ghostwhite', 'deepskyblue', 'darksalmon', 'darkcyan', 'black', 'gold', 'lightcoral', 'lightseagreen',
         'magenta', 'midnightblue', 'olive', 'orange', 'purple', 'silver', 'springgreen', 'violet']
colors = modcolors
modcolors.remove('cornflowerblue')

In [ ]:
counts_pivot.iplot(kind='line',
                    title = 'Business Categories Change over Time (All Destinations)',
                    xTitle = 'Year',
                    yTitle = 'Number of Businesses',
                    colors = colors) #full plot

In [ ]:
dropdes = dropnot[dropnot.Class != 'des']
withdes = counts_pivot
counts_pivot = dropdes.pivot('Year','Class','counts').fillna(0).astype(int) #pivot for line graph

In [ ]:
counts_pivot.iplot(kind='line',
                    title = 'Business Categories Change over Time, DES Removed',
                    xTitle = 'Year',
                    yTitle = 'Number of Businesses',
                    colors = modcolors) #full plot

In [ ]:
counts_pivot['aff'].iplot(kind='line',
                    title = 'All Fast Food Change over Time',
                    xTitle = 'Year',
                    yTitle = 'Number of Businesses',
                    color = modcolors[0]) #full plot

In [ ]:
counts_pivot['bak'].iplot(kind='line',
                    title = 'Bakeries Change over Time',
                    xTitle = 'Year',
                    yTitle = 'Number of Businesses',
                    color = modcolors[1]) #full plot

In [ ]:
counts_pivot['bar'].iplot(kind='line',
                    title = 'Bars Change over Time',
                    xTitle = 'Year',
                    yTitle = 'Number of Businesses',
                    color = modcolors[2]) #full plot

In [ ]:
counts_pivot['bds'].iplot(kind='line',
                    title = 'Bodegas Change over Time',
                    xTitle = 'Year',
                    yTitle = 'Number of Businesses',
                    color = modcolors[3]) #full plot

In [ ]:
counts_pivot['bnk'].iplot(kind='line',
                    title = 'Banks Change over Time',
                    xTitle = 'Year',
                    yTitle = 'Number of Businesses',
                    color = modcolors[4]) #full plot

In [ ]:
counts_pivot['cnv'].iplot(kind='line',
                    title = 'Convenience Stores Change over Time',
                    xTitle = 'Year',
                    yTitle = 'Number of Businesses',
                    color = modcolors[5]) #full plot

In [ ]:
counts_pivot['crd'].iplot(kind='line',
                    title = 'Credit Unions Change over Time',
                    xTitle = 'Year',
                    yTitle = 'Number of Businesses',
                    color = modcolors[6]) #full plot

In [ ]:
counts_pivot['dds'].iplot(kind='line',
                    title = 'Dental Care Facilities Change over Time',
                    xTitle = 'Year',
                    yTitle = 'Number of Businesses',
                    color = modcolors[7]) #full plot

In [ ]:
withdes['des'].iplot(kind='line',
                    title = 'Other Destinations Change over Time',
                    xTitle = 'Year',
                    yTitle = 'Number of Businesses',
                    color = 'cornflowerblue') #full plot

In [ ]:
counts_pivot['drg'].iplot(kind='line',
                    title = 'Pharmacies Change over Time',
                    xTitle = 'Year',
                    yTitle = 'Number of Businesses',
                    color = modcolors[8]) #full plot

In [ ]:
counts_pivot['eat'].iplot(kind='line',
                    title = 'Eating Places Change over Time',
                    xTitle = 'Year',
                    yTitle = 'Number of Businesses',
                    color = modcolors[9]) #full plot

In [ ]:
counts_pivot['fsh'].iplot(kind='line',
                    title = 'Fish Markets Change over Time',
                    xTitle = 'Year',
                    yTitle = 'Number of Businesses',
                    color = 'purple') #full plot

In [ ]:
counts_pivot['fvm'].iplot(kind='line',
                    title = 'Fruit and Vegetable Markets Change over Time',
                    xTitle = 'Year',
                    yTitle = 'Number of Businesses',
                    color = modcolors[11]) #full plot

In [ ]:
counts_pivot['gry'].iplot(kind='line',
                    title = 'Small Grocery Stores Change over Time',
                    xTitle = 'Year',
                    yTitle = 'Number of Businesses',
                    color = modcolors[12]) #full plot

In [ ]:
counts_pivot['hpc'].iplot(kind='line',
                    title = 'Offices or Clinics of Health Practitioners Change over Time',
                    xTitle = 'Year',
                    yTitle = 'Number of Businesses',
                    color = modcolors[13]) #full plot

In [ ]:
counts_pivot['liq'].iplot(kind='line',
                    title = 'Liquor Stores Change over Time',
                    xTitle = 'Year',
                    yTitle = 'Number of Businesses',
                    color = modcolors[14]) #full plot

In [ ]:
counts_pivot['met'].iplot(kind='line',
                    title = 'Meat Markets Change over Time',
                    xTitle = 'Year',
                    yTitle = 'Number of Businesses',
                    color = modcolors[15]) #full plot

In [ ]:
counts_pivot['mhc'].iplot(kind='line',
                    title = 'Mental Health Care Facilities Change over Time',
                    xTitle = 'Year',
                    yTitle = 'Number of Businesses',
                    color = modcolors[16]) #full plot

In [ ]:
counts_pivot['mpa'].iplot(kind='line',
                    title = 'Light/Moderate Physical Activity Facilities Change over Time',
                    xTitle = 'Year',
                    yTitle = 'Number of Businesses',
                    color = modcolors[17]) #full plot

In [ ]:
counts_pivot['mul'].iplot(kind='line',
                    title = 'Multi-use Physical Activity Facilities Change over Time',
                    xTitle = 'Year',
                    yTitle = 'Number of Businesses',
                    color = modcolors[18]) #full plot

In [ ]:
counts_pivot['nat'].iplot(kind='line',
                    title = 'Natural Food Stores Change over Time',
                    xTitle = 'Year',
                    yTitle = 'Number of Businesses',
                    color = modcolors[19]) #full plot

In [ ]:
counts_pivot['piz'].iplot(kind='line',
                    title = 'Pizza Restaurants Change over Time',
                    xTitle = 'Year',
                    yTitle = 'Number of Businesses',
                    color = modcolors[20]) #full plot

In [ ]:
counts_pivot['res'].iplot(kind='line',
                    title = 'Residential Facilities with Health Care Change over Time',
                    xTitle = 'Year',
                    yTitle = 'Number of Businesses',
                    color = modcolors[21]) #full plot

In [ ]:
counts_pivot['smk'].iplot(kind='line',
                    title = 'Large Supermarkets Change over Time',
                    xTitle = 'Year',
                    yTitle = 'Number of Businesses',
                    color = modcolors[22]) #full plot

In [ ]:
counts_pivot['urg'].iplot(kind='line',
                    title = 'Urgent Care Facilities Change over Time',
                    xTitle = 'Year',
                    yTitle = 'Number of Businesses',
                    color = modcolors[23]) #full plot

In [ ]:
counts_pivot['vpa'].iplot(kind='line',
                    title = 'Vigorous Physical Activity Facilities Change over Time',
                    xTitle = 'Year',
                    yTitle = 'Number of Businesses',
                    color = modcolors[24]) #full plot

In [ ]:
counts_pivot['war'].iplot(kind='line',
                    title = 'Warehouse and Discount Department Stores Selling Food Change over Time',
                    xTitle = 'Year',
                    yTitle = 'Number of Businesses',
                    color = modcolors[25]) #full plot

In [ ]:
counts_pivot[['aff', 'piz', 'eat']].iplot(kind='line',
                    title = 'All Restaurants over Time',
                    xTitle = 'Year',
                    yTitle = 'Number of Businesses')

In [ ]:
counts_pivot[['gry', 'smk', 'war', 'fsh', 'fvm', 'nat', 'bak', 'met', 'cnv', 'bds']].iplot(kind='line',
                    title = 'All Food Retailers over Time',
                    xTitle = 'Year',
                    yTitle = 'Number of Businesses')

In [ ]:
counts_pivot['food_retailers'] = counts_pivot['gry'] + counts_pivot['smk'] + counts_pivot['war'] + counts_pivot['fsh'] + \
counts_pivot['fvm'] + counts_pivot['nat'] + counts_pivot['bak'] + counts_pivot['met'] + counts_pivot['cnv'] + counts_pivot['bds']

In [ ]:
counts_pivot['food_retailers'].iplot(kind='line',
                    title = 'All Food Retailers (Combined) Over Time',
                    xTitle = 'Year',
                    yTitle = 'Number of Businesses')

# Individual State Investigation

We'll attempt to take a closer look at the trends shown above by looking more closely at the food environment in a few representative states

- New York
- Florida 
- California
- Texas
- Vermont
- Illinois 
- Kansas
- Colorado

In [ ]:
infile = "C:\Users\jc4673\Documents\Columbia\NETS2013_Classifications.h5"

In [ ]:
store = pd.HDFStore(infile, mode='r')
store

In [ ]:
%%time
ind = store['CA'].index
CAclass = store.select('Classifications', "columns=['DunsNumber', 'FirstYear', 'LastYear', 'Class', 'BEH_Class'] & DunsNumber=ind")

In [ ]:
CA = store. select('Locations', where="'PrimState' == 'CA' & columns=['PrimState']")

In [ ]:
CA = CA.reset_index().drop_duplicates(subset='DunsNumber', keep='last').set_index('DunsNumber')
del CA['FirstYear']

In [ ]:
store.close()

In [ ]:
store = pd.HDFStore(infile, mode='a')

In [ ]:
store.append('CA', CA, format='table', data_columns=True, index=False)

In [ ]:
store.close()

In [ ]:
%xdel CA

In [ ]:
classif = pd.read_hdf(infile, key='Classifications', columns=['LastYear', 'Class', 'BEH_Class'], chunksize=10**6)

In [ ]:
store.open()

In [ ]:
%%time
df = pd.DataFrame()
for chunk in classif:
    df = pd.concat([df, store['CA'].join(chunk, how='inner')]) 
    #DunsNumber not included for some reason

In [ ]:
df

In [ ]:
df_long = pd.read_table(infile_long, usecols=['DunsNumber', 'Year', 'Class'], index_col=['DunsNumber', 'Year'], chunksize=10**6)
df_normal = pd.read_table(infile_normal, index_col=['DunsNumber', 'FirstYear'], chunksize=10**6)
df_loc = pd.read_table(location_filename, index_col=['DunsNumber'], chunksize=10**6)

In [ ]:
loc_chunk = df_loc.get_chunk()
loc_chunk = loc_chunk.groupby(loc_chunk.index).first()

In [ ]:
allstates_chunk = loc_chunk[loc_chunk['PrimState'].isin(['NY', 'FL', 'CA', 'TX', 'VT', 'IL', 'KS', 'CO'])]

In [ ]:
cal_chunk = loc_chunk[loc_chunk['PrimState'] == 'CA']

In [ ]:
for chunk in df_loc:
    first = chunk.groupby(chunk.index).first()
    cal_chunk = cal_chunk.append(first[first['PrimState'] == 'CA'])
cal_chunk = cal_chunk.groupby(cal_chunk.index).first()

In [ ]:
cal_chunk = cal_chunk.groupby(cal_chunk.index).first()

In [ ]:
cal_chunk = cal_chunk['PrimState']
cal_chunk

In [ ]:
%%time
long_chunk = df_long.get_chunk()

In [ ]:
%%time
CA_long = long_chunk.join(cal_chunk, how='inner')

In [ ]:
%%time
for chunk in df_long:
    CA_long = CA_long.append(chunk.join(cal_chunk, how='inner'))

In [ ]:
CAcounts = CA_long.reset_index(drop=False).groupby(['Year', 'Class']).count().reset_index(drop=False)
del CAcounts['PrimState']

In [ ]:
CAcounts = CAcounts[CAcounts['Class'] != 'not']
CAcounts

In [ ]:
counts_pivot = CAcounts.pivot('Year','Class','DunsNumber').fillna(0).astype(int) #pivot for line graph
counts_pivot

In [ ]:
counts_pivot.iplot(kind='line',
                    title = 'Business Categories Change over Time (All Destinations)',
                    xTitle = 'Year',
                    yTitle = 'Number of Businesses') #full plot

In [ ]:
counts_pivot[['gry', 'smk', 'war', 'fsh', 'fvm', 'nat', 'bak', 'met', 'cnv', 'bds']].iplot(kind='line',
                    title = 'All Food Retailers over Time',
                    xTitle = 'Year',
                    yTitle = 'Number of Businesses')

In [ ]:
counts_pivot[['gry', 'smk']].iplot(kind='line',
                    title = 'All Food Retailers over Time',
                    xTitle = 'Year',
                    yTitle = 'Number of Businesses')

In [ ]:
from bisect import bisect_left

def takeClosest(myList, myNumber):
    """
    Assumes myList is sorted. Returns closest value to myNumber.

    If two numbers are equally close, return the smallest number.
    """
    pos = bisect_left(myList, myNumber)
    if pos == 0:
        return myList[0]
    if pos == len(myList):
        return myList[-1]
    before = myList[pos - 1]
    after = myList[pos]
    if after - myNumber < myNumber - before:
       return after
    else:
       return before

In [ ]:
# Select all CA duns from the current chunk
%timeit CAchunk = long_chunk.loc[CAduns[:CAduns.index(takeClosest(CAduns, long_chunk.index.get_level_values(0)[-1]))]]

In [ ]:
indices = CAduns[:CAduns.index(takeClosest(CAduns, long_chunk.index.get_level_values(0)[-1]))]

In [ ]:
CAchunk = long_chunk.at[CAduns[:CAduns.index(takeClosest(CAduns, long_chunk.index.get_level_values(0)[-1]))]]